#  Call Dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import image

from keras.models import Sequential,Model,load_model
from keras.layers import Input, Dense, Activation, Dropout, LSTM, Flatten, Embedding, merge,TimeDistributed,Concatenate,Bidirectional,BatchNormalization,Add,AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,Lambda, Permute,Reshape,Multiply,multiply
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import metrics
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras import optimizers

from keras_bert import get_custom_objects
from tqdm import tqdm
from chardet import detect
import keras
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import Tokenizer as k_Tokenizer

import sklearn
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,LabelBinarizer
from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score,roc_curve,confusion_matrix,auc
from sklearn.utils import class_weight
from  sklearn.metrics import precision_recall_fscore_support


import os, argparse,cv2,h5py,string
import collections
from collections import Counter


from scipy import interp
from itertools import cycle
import seaborn as sns

from deepexplain.tensorflow import DeepExplain
import codecs

import innvestigate
import innvestigate.utils as iutils

from skimage.measure import compare_ssim as ssim
from skimage import data
from skimage.color import rgb2gray

from tensorflow.compat.v1 import ConfigProto,InteractiveSession
from keras.backend.tensorflow_backend import set_session,clear_session,get_session



config = tf.ConfigProto()
config.gpu_options.visible_device_list = "0"
config.gpu_options.per_process_gpu_memory_fraction =.3
set_session(tf.Session(config=config))

# config = tf.ConfigProto()
# config.gpu_options.visible_device_list = "1"
# config.gpu_options.per_process_gpu_memory_fraction = .3
# set_session(tf.Session(config=config))

# G =tf.Graph()
# sess1 = tf.Session(graph=G, config=tf.ConfigProto(log_device_placement=False,gpu_options=tf.GPUOptions(allow_growth=True,visible_device_list='0')))
# sess2 = tf.Session(graph=G, config=tf.ConfigProto(log_device_placement=False,gpu_options=tf.GPUOptions(allow_growth=True,visible_device_list='1')))



# Seed value

# Apparently you may use different seed values at each stage
seed_value= 2019

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.set_random_seed(seed_value)


# Declare the two file name


Using TensorFlow backend.
/home/local/AD/psanjay/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
W0508 15:52:49.730612 140587179886400 deprecation_wrapper.py:119] From /home/local/AD/psanjay/src/deepexplain/deepexplain/tensorflow/methods.py:556: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [2]:
os.getcwd()

'/home/local/AD/asarkar2/Research/Modality'

#  Call the files

In [3]:
### This files contains the Image ID, Related Question and Corrospoding answers. Each file contains training data (first 3200), validation data (next 500) and testing data (rest 125)

######################## If we do analysis for "Modality" category ############################


filename_mod="/home/local/AD/asarkar2/Question/C1_Modality_all.txt"
filename_glove="/home/local/AD/asarkar2/glove.6B.100d.txt"     ### golve vector

# filename_organ="/home/local/AD/asarkar2/Question/C3_Organ_all.txt"
# filename_plane="/home/local/AD/asarkar2/Question/C2_plane_all.txt"

### This files are uploaded to "Data file" in google drive
### Data File------> All Question  (Location)

####################### Image Path #################################################################

### This files are uploaded to "Data file" in google drive
### Data File------> All Image  (Location)

img_path_train="/home/local/AD/asarkar2/Image File/VQAMed2019_Train_Images/VQAMed2019_Train_Images"
img_path_valid="/home/local/AD/asarkar2/Image File/VQAMed2019_Valid_Images/VQAMed2019_Valid_Images"
img_path_test="/home/local/AD/asarkar2/Image File/VQAMed2019_Test_Images/VQAMed2019_Test_Images"






In [4]:
os.getcwd()

'/home/local/AD/asarkar2/Research/Modality'

#  Call a "separate" function to separate image id, question and answer from above file

In [5]:
%run model_util.ipynb

In [6]:
img_id=separate(filename_mod).get("Image_id")
question=separate(filename_mod).get("Question")
answer=separate(filename_mod).get("Answer")




In [7]:
# first 3200 elements are from train data
# 3200-3700 elements are from valid data
# rest are from test data

image_id_train=img_id[:3200]
image_id_valid=img_id[3200:3700]
image_id_test=img_id[3700:]


question_train=question[:3200]
question_valid=question[3200:3700]
question_test=question[3700:]

answer_train=answer[:3200]
answer_valid=answer[3200:3700]
answer_test=answer[3700:]


In [8]:
final_answer_valid=[items if items in answer_train else "unknown" for items in answer_valid]

In [9]:
final_answer_test=[items if items in answer_train else "unknown" for items in answer_test]

In [10]:
final_answer_train=answer_train

# Call image_preprocess function to process the image file

In [12]:
train_image=image_preprocess(img_path_train,image_id_train)
valid_image=image_preprocess(img_path_valid,image_id_valid)
test_image=image_preprocess(img_path_test,image_id_test)


In [13]:
# train_image=np.load("./Image_array/modality_train_image.npy")
# valid_image=np.load("./Image_array/modality_valid_image.npy")
# test_image=np.load("./Image_array/modality_test_image.npy")



In [14]:
# lrp_train=np.load("mask_dtd_organ_resnet_lstm_train.npy")
# lrp_valid=np.load("mask_dtd_organ_resnet_lstm_valid.npy")
# lrp_test=np.load("mask_dtd_organ_resnet_lstm_test.npy")


# lrp_train=np.array(lrp_train)
# lrp_valid=np.array(lrp_valid)
# lrp_test=np.array(lrp_test)

#  Standardized the image pixel

In [15]:
train_image_std=train_image/255
valid_image_std=valid_image/255
test_image_std=test_image/255

#  Call label_preprocess Class to pre-process the answer (label)

In [17]:


################# Integer Label Encoding ##########################################

integer_label_train=LabelPreprocess.integer_encoding(final_answer_train,final_answer_train)
integer_label_valid=LabelPreprocess.integer_encoding(final_answer_valid,final_answer_train)
integer_label_test=LabelPreprocess.integer_encoding(final_answer_test,final_answer_train)




In [18]:
################# One-Hot Label Encoding ##########################################


onehot_label_train=LabelPreprocess.onehot_encoding(final_answer_train,final_answer_train)
onehot_label_valid=LabelPreprocess.onehot_encoding(final_answer_valid,final_answer_train)
onehot_label_test=LabelPreprocess.onehot_encoding(final_answer_test,final_answer_train)


/home/local/AD/psanjay/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/local/AD/psanjay/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify

#  BERT model question pre-process

In [19]:
# %run bert_question_preprocess.ipynb

In [20]:
# train_ques_bert,valid_ques_bert,test_ques_bert=bert_question_preprocess(13,'/home/local/AD/asarkar2/BERT',question)

#  LSTM model question pre-process

In [22]:
############################## sequence of the token for question ############################################

seq_train_question=QuestionPreprocess.sequence_question(question_train,question_train)
seq_valid_question=QuestionPreprocess.sequence_question(question_valid,question_train)
seq_test_question=QuestionPreprocess.sequence_question(question_test,question_train)


In [23]:
# lenght=[]
# for i in range(len(seq_train_question)):
#     l=seq_train_question[i]
#     le=len(l)
#     lenght.append(le)

In [24]:
# np.max(lenght)

In [25]:
############################### Padding question ########################################################

# maxlength of each question for modality-----> 11
# maxlength of each question for organ-----> 10
# maxlength of each question for modality-----> 9


padding_train_ques=QuestionPreprocess.padding_question(seq_train_question,maxlength=11,padding_criterion="post")
padding_valid_ques=QuestionPreprocess.padding_question(seq_valid_question,maxlength=11,padding_criterion="post")
padding_test_ques=QuestionPreprocess.padding_question(seq_test_question,maxlength=11,padding_criterion="post")


In [26]:
padding_test_ques.shape

(125, 11)

#  Embedding Matrix for Glove Vector

In [ ]:
def embedding(glove_filename,token_data): 
    word_index=token_data # call my_token function
    embeddings_index = dict()
    f = open(glove_filename,encoding='utf8') # call glove vector text file
    for line in f:
        values=line.split() # split the each line in text file
        word = values[0] # first index associate with word and othe other indexs represent embedding vector associated with that word 
        coefs = np.asarray(values[1:], dtype='float32') 
        embeddings_index[word] = coefs
#         print(embeddings_index)
    f.close()

    vocab_size=len(token_data)
    embedding_matrix=np.zeros((vocab_size+1,100)) # define embedding matrix
#     print(word_index.items())
    for word,i in word_index.items():                # create our embedding matrix for each word of questions.
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        elif embedding_vector is None:
            embedding_matrix[i] =embeddings_index.get("unk")
    return embedding_matrix


In [ ]:
token_data=QuestionPreprocess.tokenize_question(question_train)

emb_mat=embedding(filename_glove,token_data)

# VQA Model 

In [ ]:
# %run vgg_model_mfb.ipynb
%run resnet_model_mfb.ipynb
%run lstm_text_model.ipynb
# %run bert_text_model.ipynb


In [ ]:
########################### RESNET-50 & LSTM ####################

resnet=resnet_image_model()
lstm=lstm_text_model(emb_mat,emb_mat.shape[0],padding_train_ques.shape[1])

# combined = Concatenate(axis=1)([lstm.output,resnet.output])
# c1=Dense(2048,activation="relu")(combined)
# c2=Dense(1000,activation="relu")(c1)
# c3=Dense(onehot_label_train.shape[1])(c2)
# c4=Activation("softmax")(c3)

# model_vqa = Model(inputs=[lstm.input,resnet.input], outputs=c4)


# MFB

In [ ]:
####

lstm_out=lstm.output


'''
Question Attention
'''

qatt_conv1=Convolution2D(2048,(1,1))(lstm.output)
qatt_relu = Activation("relu")(qatt_conv1)
qatt_conv2 = Convolution2D(2,(1,1)) (qatt_relu) # (N,L,1,2)
qatt_conv2=Lambda(lambda x: K.squeeze(x, axis=2))(qatt_conv2)
qatt_conv2 = Permute((2,1))(qatt_conv2)

qatt_softmax = Activation("softmax")(qatt_conv2)
qatt_softmax =Reshape( (11, 1,2))(qatt_softmax)


def t_qatt_mask(tensors):
#     lstm=lstm_text_model(emb_mat,emb_mat.shape[0],padding_train_ques.shape[1])
#     lstm_out=lstm.output
    qatt_feature_list = []
    ten1=tensors[0]
    ten2=tensors[1]

    for i in range(2):
        t_qatt_mask = ten1[:,:,:,i]  # (N,1,L,1)
        t_qatt_mask=K.reshape(t_qatt_mask,(-1,11,1,1))
        t_qatt_mask = t_qatt_mask * ten2  # (N,1024,L,1)
#         print(t_qatt_mask)
    #     t_qatt_mask = K.sum(t_qatt_mask,axis=1,keepdims=True)
        t_qatt_mask= K.sum(t_qatt_mask,axis=1,keepdims=True)
#         print(t_qatt_mask)

        qatt_feature_list.append(t_qatt_mask)
    
    qatt_feature_concat = K.concatenate(qatt_feature_list)  # (N,2048,1,1)
    return qatt_feature_concat


q_feat_resh=Lambda(t_qatt_mask)([qatt_softmax,lstm_out])

q_feat_resh =Lambda(lambda x: K.squeeze(x, axis=2))(q_feat_resh)

q_feat_resh=Permute((2,1))(q_feat_resh)
q_feat_resh=Reshape([2048])(q_feat_resh)                                           # (N,2048)




'''
 MFB Image  with Attention

'''

# image_feature=vgg.output


i_feat_resh = Reshape((9,1, 2048))(resnet.output)  # (N,2048,9)

iatt_fc = Dense(5000,activation="tanh")(q_feat_resh)  # (N,5000)
iatt_resh = Reshape( (1, 1,5000))(iatt_fc)  # (N,5000,1,1)
iatt_conv = Convolution2D(5000,(1,1)) (i_feat_resh) # (N,5000,9,1)

iatt_eltwise = multiply([iatt_resh , iatt_conv])  # (N,5000,9,1)
iatt_droped = Dropout(0.1)(iatt_eltwise)

iatt_permute1 = Permute(( 3 ,2, 1))(iatt_droped)  # (N,9,5000,1)
iatt_resh2 = Reshape( (1000, 5,9))(iatt_permute1)
iatt_sum =Lambda(lambda x: K.sum(x, axis=2,keepdims=True))(iatt_resh2)
iatt_permute2 =Permute((3, 2, 1))(iatt_sum)  # (N,1000,9,1)
iatt_sqrt = Lambda(lambda x:K.sqrt(Activation("relu")(x)) - K.sqrt(Activation("relu")(-x)))(iatt_permute2)
iatt_sqrt =Reshape([-1])(iatt_sqrt)
iatt_l2=Lambda(lambda x: K.l2_normalize(x,axis=1))(iatt_sqrt)
iatt_l2=Reshape((9,1,1000))(iatt_l2)


iatt_conv1 =Convolution2D (2048,(1,1)) (iatt_l2) # (N,2048,9,1)
iatt_relu = Activation("relu")(iatt_conv1)
iatt_conv2 = Convolution2D(2,(1,1))(iatt_relu)  # (N,2,9,1)
iatt_conv2 = Reshape((2,9) )(iatt_conv2)
iatt_softmax = Activation("softmax")(iatt_conv2)
iatt_softmax = Reshape((9,1, 2))(iatt_softmax)


def iatt_feature_list(tensors):
#     global i_feat_resh
    ten3=tensors[0]
    ten4=tensors[1]
    iatt_feature_list = []
    for j in range(2):
        iatt_mask = ten3[:,:,:,j] # (N,1,9,1)
        iatt_mask=K.reshape(iatt_mask,(-1,9,1,1))   
        iatt_mask = iatt_mask * ten4  # (N,2048,9,1)
#         print(iatt_mask)
        iatt_mask = K.sum(iatt_mask,axis=1 ,keepdims=True)
        iatt_feature_list.append(iatt_mask)
    iatt_feature_cat = K.concatenate(iatt_feature_list)  # (N,1024,1,1)
    return iatt_feature_cat

iatt_feature_cat=Lambda(iatt_feature_list)([iatt_softmax,i_feat_resh])
iatt_feature_cat =Lambda(lambda x: K.squeeze(x, axis=2))(iatt_feature_cat)
iatt_feature_cat=Permute((2,1))(iatt_feature_cat)
iatt_feature_cat=Reshape([-1])(iatt_feature_cat)


In [ ]:
'''
Fine-grained Image-Question MFH fusion
'''
# print(q_feat_resh.shape)
# print(bert_encode.shape)

# if mode != 'train':
#     q_feat_resh = q_feat_resh.unsqueeze(0)

mfb_q = Dense(5000, activation="tanh")(q_feat_resh)  # (N,5000)
mfb_i = Dense(5000,activation="relu")(iatt_feature_cat)  # (N,5000)
mfb_eltwise =multiply([mfb_q, mfb_i])
mfb_drop1 = Dropout(0.1)(mfb_eltwise)
mfb_resh = Reshape(  (1000,5,1))(mfb_drop1)  # (N,1,1000,5)
mfb_sum = Lambda(lambda x: K.sum(x, axis=2, keepdims=True))(mfb_resh)
mfb_out = Reshape([1000])(mfb_sum)
mfb_sqrt =Lambda(lambda x: K.sqrt(Activation("relu")(x)) - K.sqrt(Activation("relu")(-x)))(mfb_out)

# if mode != 'train':
#     mfb_sqrt = mfb_sqrt.unsqueeze(0)

mfb_l2_1 = Lambda(lambda x: K.l2_normalize(x,axis=1))(mfb_sqrt)

mfb_q2 = Dense(5000, activation="tanh")(q_feat_resh) # (N,5000)
mfb_i2 = Dense(5000,activation="relu")(iatt_feature_cat)  # (N,5000)
mfb_eltwise2 = multiply([mfb_q2, mfb_i2])  # (N,5000)
mfb_eltwise2 =multiply([mfb_eltwise2, mfb_drop1])
mfb_drop2 = Dropout(0.1)(mfb_eltwise2)
mfb_resh2 = Reshape( (1000,5,1))(mfb_drop2)
#                              # (N,1,1000,5)
mfb_sum2 = Lambda(lambda x: K.sum(x, 2, keepdims=True))(mfb_resh2)
mfb_out2 =Reshape([1000])(mfb_sum2)
mfb_sqrt2 =Lambda(lambda x: K.sqrt(Activation("relu")(x)) - K.sqrt(Activation("relu")(-x)))(mfb_out2)

# if mode != 'train':
#     mfb_sqrt2 = mfb_sqrt2.unsqueeze(0)
mfb_l2_2 =Lambda(lambda x: K.l2_normalize(x,axis=1))(mfb_sqrt2)

mfb_l2_3 = Concatenate(axis=1)([mfb_l2_1, mfb_l2_2])  # (N,2000)
prediction = Dense(onehot_label_train.shape[1],activation="softmax")(mfb_l2_3)

# # prediction=F.log_softmax(prediction)
# # prediction=self.Sigmoid(prediction)




In [ ]:
model_vqa=Model(inputs=[lstm.input,resnet.input],outputs=prediction)


In [ ]:
model_vqa.summary()

#  Model Compile

In [ ]:
sgd = optimizers.SGD(lr=0.01, momentum=0.99)

In [ ]:
model_vqa.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])


In [ ]:
# class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(answer[:3700]),
#                                                  answer[:3700])

#  Train Model

In [ ]:
#################################### VGG 16 & LSTM ###########################################################
callback = [EarlyStopping(monitor='val_loss',mode="min",verbose=1, patience=30),
             ModelCheckpoint('train_{}_{}_{}_{}.h5'.format("resnet","lstm","mfb_coatt","modality"), monitor='val_loss',mode="min" ,verbose=1,save_best_only=True)]
fit=model_vqa.fit([padding_train_ques,train_image_std],onehot_label_train,validation_data=([padding_valid_ques,valid_image_std],onehot_label_valid),epochs=300,batch_size=32,verbose=0, callbacks=callback)

In [ ]:
# #################################### VGG 16 & BERT ###########################################################
# callback = [EarlyStopping(monitor='val_loss',mode="min",verbose=1, patience=40),
#              ModelCheckpoint('train_{}_{}_{}.h5'.format("resnet","bert","mod"), monitor='val_loss',mode="min" ,verbose=1,save_best_only=True)]
# fit=model_vqa.fit(train_ques_bert+[train_image_std],onehot_label_train,validation_data=(valid_ques_bert+[valid_image_std],onehot_label_valid),epochs=300,batch_size=32,verbose=0, callbacks=callback)

In [ ]:
#plot training history
plt.plot(fit.history['acc'], label='train')
plt.plot(fit.history['val_acc'], label='test')
plt.legend()
plt.show()

#  Model Evaluation

In [30]:
### Sample for any LSTM + (VGG/RESNET) model

################### load VGG 16 & LSTM model #################################

model = load_model('./3-Model/train_resnet_lstm_mfb_coatt_modality.h5',custom_objects=get_custom_objects())

W0508 15:56:59.059647 140587179886400 deprecation_wrapper.py:119] From /home/local/AD/psanjay/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0508 15:56:59.090624 140587179886400 deprecation_wrapper.py:119] From /home/local/AD/psanjay/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0508 15:56:59.102488 140587179886400 deprecation_wrapper.py:119] From /home/local/AD/psanjay/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0508 15:56:59.129834 140587179886400 deprecation_wrapper.py:119] From /home/local/AD/psanjay/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. P

In [31]:
train_accu=model.evaluate([padding_train_ques,train_image_std],onehot_label_train)
valid_accu=model.evaluate([padding_valid_ques,valid_image_std],onehot_label_valid)
test_accu=model.evaluate([padding_test_ques,test_image_std],onehot_label_test)


125/125 [==============================] - 1s 7ms/step


In [32]:
# train_accu=model.evaluate([padding_train_ques,lrp_train],onehot_label_train)
# valid_accu=model.evaluate([padding_valid_ques,lrp_valid],onehot_label_valid)
# test_accu=model.evaluate([padding_test_ques,lrp_test],onehot_label_test)


In [33]:
print(train_accu, valid_accu,test_accu)

[0.8118198376893997, 0.701875] [1.0001466975212097, 0.6700000009536743] [1.0262185745239258, 0.6799999942779541]


#  AUROC

In [34]:
y_pred_train=model.predict([padding_train_ques,train_image_std])
y_pred_valid=model.predict([padding_valid_ques,valid_image_std])
y_pred_test=model.predict([padding_test_ques,test_image_std])


y_pred_valid1=np.argmax(y_pred_valid,axis=1)
y_pred_train1=np.argmax(y_pred_train,axis=1)
y_pred_test1=np.argmax(y_pred_test,axis=1)

# len(y_pred_valid1)

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)



ruc_train=multiclass_roc_auc_score(integer_label_train, y_pred_train1, average="macro")
ruc_valid=multiclass_roc_auc_score(integer_label_valid, y_pred_valid1, average="macro")
ruc_test=multiclass_roc_auc_score(integer_label_test,y_pred_test1, average="macro")

print(ruc_train,ruc_valid,ruc_test)

0.6542676526456089 0.671241464202495 0.7444873728838612


#  Recall, Precision & F-1


In [ ]:
def class_report(y_true, y_pred, y_score=None, average='micro'):
    if y_true.shape != y_pred.shape:
        print("Error! y_true %s is not the same shape as y_pred %s" % (
              y_true.shape,
              y_pred.shape)
        )
        return

    lb = LabelBinarizer()

    if len(y_true.shape) == 1:
        lb.fit(y_true)

    #Value counts of predictions
    labels, cnt = np.unique(
        y_pred,
        return_counts=True)
    n_classes = len(labels)
    pred_cnt = pd.Series(cnt, index=labels)

    metrics_summary = precision_recall_fscore_support(
            y_true=y_true,
            y_pred=y_pred,
            labels=labels)

    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='weighted'))

    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support']
    class_report_df = pd.DataFrame(
        list(metrics_summary),
        index=metrics_sum_index,
        columns=labels)

    support = class_report_df.loc['support']
    total = support.sum() 
    class_report_df['avg / total'] = avg[:-1] + [total]

    class_report_df = class_report_df.T
    class_report_df['pred'] = pred_cnt
    class_report_df['pred'].iloc[-1] = total

    if not (y_score is None):
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for label_it, label in enumerate(labels):
            fpr[label], tpr[label], _ = roc_curve(
                (y_true == label).astype(int), 
                y_score[:, label_it])

            roc_auc[label] = auc(fpr[label], tpr[label])

        if average == 'micro':
            if n_classes <= 2:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                    lb.transform(y_true).ravel(), 
                    y_score[:, 1].ravel())
            else:
                fpr["avg / total"], tpr["avg / total"], _ = roc_curve(
                        lb.transform(y_true).ravel(), 
                        y_score.ravel())

            roc_auc["avg / total"] = auc(
                fpr["avg / total"], 
                tpr["avg / total"])

        elif average == 'macro':
            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([
                fpr[i] for i in labels]
            ))

            # Then interpolate all ROC curves at this points
            mean_tpr = np.zeros_like(all_fpr)
            for i in labels:
                mean_tpr += interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr

            roc_auc["avg / total"] = auc(fpr["macro"], tpr["macro"])

        class_report_df['AUC'] = pd.Series(roc_auc)

    return class_report_df



In [ ]:
class_report(integer_label_test, y_pred_test1, y_score=None, average='macro')